In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import sys
import time
import lightgbm as lgb
from catboost import CatBoostClassifier
import xgboost as xgb
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from IPython.display import HTML
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split
from hyperopt import fmin, tpe, hp, anneal, Trials
import hyperopt
import gc
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
usecols = ['fecha_dato', 'ncodpers', 'ind_empleado', 'pais_residencia', 'sexo',
        'age', 'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel',
        'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext',
        'canal_entrada', 'indfall','nomprov', 'ind_actividad_cliente', 'renta', 'segmento',
       'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']

# USECOLS ARE ALL THE COLUMNS THAT WE ARE USING FOR OUR PROJECT
# WE HAVE DROPPED A TOTAL OF 4 COLUMNS WHICH WERE TOO SKEWED.

usecols1 = ['fecha_dato', 'ncodpers', 'ind_empleado', 'pais_residencia', 'sexo','age', 
            'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel','indrel_1mes', 'tiprel_1mes',
        'indresi', 'indext','canal_entrada', 'indfall','nomprov', 'ind_actividad_cliente', 'renta', 'segmento']

# THESE ARE ALL THE FEATURE COLUMNS WE ARE USING.

usecols2 = ['fecha_dato','ncodpers','ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']

# THESE ARE ALL THE PRODUCT COLUMNS .

# **DATA LOADING**

In [ ]:
# LOADING OF TRAIN AND TEST DATA IN df and df_t respectively.

df = pd.read_csv('../input/santander-pr/train.csv',dtype={"sexo":str,"ind_nuevo":str,"ult_fec_cli_1t":str,"indext":str,"ind_empleado":"category","ind_nuevo":"category","indrel":"category","indrel_1mes":"category","tiprel_1mes":"category","indresi":"category","indext":"category","conyuemp":"category","indfall":"category","canal_entrada":"category","ind_ahor_fin_ult1":"int8",'ind_aval_fin_ult1':"int8", 'ind_cco_fin_ult1':"int8",'ind_cder_fin_ult1':"int8", 'ind_cno_fin_ult1':"int8", 'ind_ctju_fin_ult1':"int8",'ind_ctma_fin_ult1':"int8", 'ind_ctop_fin_ult1':"int8", 'ind_ctpp_fin_ult1':"int8",'ind_deco_fin_ult1':"int8", 'ind_deme_fin_ult1':"int8", 'ind_dela_fin_ult1':"int8",'ind_ecue_fin_ult1':"int8", 'ind_fond_fin_ult1':"int8", 'ind_hip_fin_ult1':"int8",'ind_plan_fin_ult1':"int8", 'ind_pres_fin_ult1':"int8", 'ind_reca_fin_ult1':"int8",'ind_tjcr_fin_ult1':"int8", 'ind_valo_fin_ult1':"int8", 'ind_viv_fin_ult1':"int8", 'ind_recibo_ult1':"int8"}, usecols=usecols)
df_t = pd.read_csv('../input/santander-pr/test.csv',dtype={"sexo":str,"ind_nuevo":str,"ult_fec_cli_1t":str,"indext":str,"ind_empleado":"category","ind_nuevo":"category","indrel":"category","indrel_1mes":"category","tiprel_1mes":"category","indresi":"category","indext":"category","conyuemp":"category","indfall":"category","canal_entrada":"category"}, usecols=usecols1)

In [ ]:
# HERE WE ARE SORTING OUR DATA FOR USING LAGS AND TOGGLE FEATURES

df.sort_values(by=['ncodpers','fecha_dato'],inplace=True)

# **PRE-PROCESSING**

**DROPPING OF ROWS AND ORGANISING COLUMNS**

In [ ]:
# WE Dropped around 27734 rows which had all its columns empty.

df=df[df['indrel'].notna()]
df=df.dropna(subset=['ind_empleado'])

# Here we are converting our string dates into datetime datatype . 

#TRAIN
df["fecha_dato"] = pd.to_datetime(df["fecha_dato"],format="%Y-%m-%d")
df["fecha_alta"] = pd.to_datetime(df["fecha_alta"],format="%Y-%m-%d")

#TEST
df_t["fecha_dato"] = pd.to_datetime(df_t["fecha_dato"],format="%Y-%m-%d")
df_t["fecha_alta"] = pd.to_datetime(df_t["fecha_alta"],format="%Y-%m-%d")

# We are dropping antiguedad which refers to the seniority of a customer but rather we are using
# a new column cust_since which is the numeric difference between fecha_alta and fecha_dato since antiguedad had
# a lot of weird values.

# TRAIN
df.drop(['antiguedad'],axis=1,inplace=True)
df['cust_since'] = df['fecha_dato'].dt.to_period('M').astype(int) - df['fecha_alta'].dt.to_period('M').astype(int)
# TEST
df_t.drop(['antiguedad'],axis=1,inplace=True)
df_t['cust_since'] = df_t['fecha_dato'].dt.to_period('M').astype(int) - df_t['fecha_alta'].dt.to_period('M').astype(int)

# In column indrel_1mes the data was not uniform for similar data, some in integer some in string so we converted
# it using a dictionary.
map_dict = { 1.0  : "1",
            "1.0" : "1",
            "1"   : "1",
            "3.0" : "3",
            "P"   : "P",
            3.0   : "3",
            2.0   : "2",
            "3"   : "3",
            "2.0" : "2",
            4.0   : "4",
            "4.0" : "4",
            "4"   : "4",
            "2"   : "2"}

# TRAIN

df.indrel_1mes = df.indrel_1mes.apply(lambda x: map_dict.get(x,x))
# TEST

df_t.indrel_1mes = df_t.indrel_1mes.apply(lambda x: map_dict.get(x,x))

**REPLACING OF NULL VALUES**

In [ ]:
#We replaced the null values using 1

df.indrel_1mes.fillna("1",inplace=True)  # TRAIN
df_t.indrel_1mes.fillna("1",inplace=True) # TEST

# FILLING OF NULL VALUES

# WE HAVE FILLED THE NULL VALUES OF ALL COLUMNS USING THE MODE OF THE COLUMN.

# TRAIN

df['ind_empleado'].fillna("N",inplace=True)
df['pais_residencia'].fillna("ES",inplace=True)
df['indresi'].fillna("S",inplace=True)
df['indext'].fillna("N",inplace=True)
df['indfall'].fillna("N",inplace=True)
df['sexo'].fillna(df['sexo'].mode()[0], inplace=True)
df['segmento'].fillna(df['segmento'].mode()[0], inplace=True)
df['canal_entrada'] = df['canal_entrada'].astype('str')
df['canal_entrada'].fillna("UNK",inplace=True)
df.tiprel_1mes.fillna("A",inplace=True)
df['ind_nomina_ult1'].fillna(0,inplace=True)
df['ind_nom_pens_ult1'].fillna(0,inplace=True)
df.tiprel_1mes.replace(to_replace='N',value='A',inplace=True)

# TEST

df_t['ind_empleado'].fillna("N",inplace=True)
df_t['pais_residencia'].fillna("ES",inplace=True)
df_t['indresi'].fillna("S",inplace=True)
df_t['indext'].fillna("N",inplace=True)
df_t['indfall'].fillna("N",inplace=True)
df_t['sexo'].fillna(df_t['sexo'].mode()[0], inplace=True)
df_t['segmento'].fillna(df_t['segmento'].mode()[0], inplace=True)
df_t['canal_entrada'] = df_t['canal_entrada'].astype('str')
df_t['canal_entrada'].fillna("UNK",inplace=True)
df_t.tiprel_1mes.fillna("A",inplace=True)
df_t.tiprel_1mes.replace(to_replace='N',value='A',inplace=True)

**HANDLING OF NUMERICAL COLUMNS (REMOVING OUTLIERS AND NORMALISING)**

In [ ]:
#NOW WE WILL HANDLE THE NUMERICAL COLUMNS OF RENTA,AGE and CUST_SINCE.

# FIRST THE RENTA COLUMN.

# FOR REPLACING THE NULL VALUES OF RENTA WE GROUP IT WITH THEIR RESPECTIVE PROVINCE'S MEAN.
# AFTER THAT WE REMOVE OUTLIERS AT 75% QUANTILE MULTIPLIED BY 4 AND 25% QUANTILE DIVIDED BY 4.
# THEN WE STANDARDISE THE COLUMN.

# TRAIN

df['nomprov'].fillna(df['nomprov'].mode()[0], inplace=True)
df['renta']=df[['nomprov','renta']].groupby(['nomprov']).transform(lambda x: x.fillna(x.median())).renta
max_out=df.renta.quantile(0.75)*4
min_out=df.renta.quantile(0.25)/4
df.loc[df['renta']>max_out,'renta']=max_out
df.loc[df['renta']<min_out,'renta']=min_out
scale=StandardScaler().fit(df[['renta']])
df[['renta']]=scale.transform(df[['renta']])

# TEST 

df_t['nomprov'].fillna(df_t['nomprov'].mode()[0], inplace=True)
df_t['renta']=df_t[['nomprov','renta']].groupby(['nomprov']).transform(lambda x: x.fillna(x.median())).renta
max_out=df_t.renta.quantile(0.75)*4
min_out=df_t.renta.quantile(0.25)/4
df_t.loc[df_t['renta']>max_out,'renta']=max_out
df_t.loc[df_t['renta']<min_out,'renta']=min_out
scale=StandardScaler().fit(df_t[['renta']])
df_t[['renta']]=scale.transform(df_t[['renta']])


# SECOND THE AGE COLUMN
# WE REMOVE OUTLIERS OF AGES >95 and <12.
# THEN WE USE NORMALIZATION , MIN-MAX TECHNIQUE.

# TRAIN

df.age = pd.to_numeric(df.age,errors="coerce")
max_out=df.age.quantile(0.75)*1.9
min_out=df.age.quantile(0.25)/2
df.loc[df['age']>max_out,'age']=max_out
df.loc[df['age']<min_out,'age']=min_out
scale=MinMaxScaler().fit(df[['age']])
df[['age']]=scale.transform(df[['age']])

# TEST

df_t.age = pd.to_numeric(df_t.age,errors="coerce")
max_out=df_t.age.quantile(0.75)*1.9
min_out=df_t.age.quantile(0.25)/2
df_t.loc[df_t['age']>max_out,'age']=max_out
df_t.loc[df_t['age']<min_out,'age']=min_out
scale=MinMaxScaler().fit(df_t[['age']])
df_t[['age']]=scale.transform(df_t[['age']])


# THIRD THE CUST_SINCE COLUMN
# WE NORMALISE THE COLUMNS, MIN-MAX TECHNIQUE.

# TRAIN

scale=MinMaxScaler().fit(df[['cust_since']])
df[['cust_since']]=scale.transform(df[['cust_since']])

# TEST

scale=MinMaxScaler().fit(df_t[['cust_since']])
df_t[['cust_since']]=scale.transform(df_t[['cust_since']])

**CHANGING DATA TYPES**

In [ ]:
# NOW WE CHANGE DATA TYPES OF COLUMNS FOR BETTER RAM MANAGEMENT.

# TRAIN

df.ind_nuevo=df.ind_nuevo.astype(np.float64)
df.indrel=df.indrel.astype(np.float64)
df.ind_actividad_cliente=df.ind_actividad_cliente.astype(np.int8)
df.ind_nuevo=df.ind_nuevo.astype(np.int8)
df.indrel=df.indrel.astype(np.int8)
df.ind_nomina_ult1=df.ind_nomina_ult1.astype(np.int8)
df.ind_nom_pens_ult1=df.ind_nom_pens_ult1.astype(np.int8)
df.ind_empleado=df.ind_empleado.astype("category")
df.sexo=df.sexo.astype("category")

# TEST

df_t.ind_nuevo=df_t.ind_nuevo.astype(np.float64)
df_t.indrel=df_t.indrel.astype(np.float64)
df_t.ind_actividad_cliente=df_t.ind_actividad_cliente.astype(np.int8)
df_t.ind_nuevo=df_t.ind_nuevo.astype(np.int8)
df_t.indrel=df_t.indrel.astype(np.int8)
df_t.ind_empleado=df_t.ind_empleado.astype("category")
df_t.sexo=df_t.sexo.astype("category")

**REDUCING FEATURES IN CATEGORICAL COLUMNS**

In [ ]:

# IN MANY COLUMNS,WE HAD A LOT OF FEATURES AND IT WAS DIFFICULT TO ONE-HOT ENCODE THEM SO WE REDUCED THE NUMBER
# OF FEATURES IN COLUMN TO BE FINITE IN 2 COLUMNS.

# COLUMN CANAL_ENTRADA
# WE REDUCED THE NUMBER OF FEATURES TO BE THE 10 MOST POPULAR FEATURES AND ALL THE REST TO BE IN OTH.

canal_name={'KHE' : 'KHE',
            'KAT' : 'KAT',
            'KFC' : 'KFC',
            'KHQ' : 'KHQ',
            'KFA' : 'KFA',
            'KHK' : 'KHK',
            'KHM' : 'KHM',
            'UNK' : 'UNK',
            'KHD' : 'KHD',
            'KHN' : 'KHN'}
arr=list(df['canal_entrada'].unique())

for name in arr:
    if(name not in canal_name.keys()):
        canal_name[name]="OTH"

arr=list(df_t['canal_entrada'].unique()) 

for name in arr:
    if(name not in canal_name.keys()):
        canal_name[name]="OTH"

df.canal_entrada = df.canal_entrada.apply(lambda x: canal_name.get(x,x))  # TRAIN

df_t.canal_entrada = df_t.canal_entrada.apply(lambda x: canal_name.get(x,x))  # TEST

# COLUMN NOMPROV
# WE REDUCED THE NUMBER OF FEATURES TO BE THE 10 MOST POPULAR FEATURES AND ALL THE REST TO BE IN OTH.

city_name={'MADRID' : 'MADRID',
           'BARCELONA' : 'BARCELONA',
           'VALENCIA' : 'VALENCIA',
           'SEVILLA' : 'SEVILLA',
           'CORUÑA, A' : 'CORUNA',
           'MURCIA' : 'MURCIA',
           'MALAGA' : 'MALAGA',
           'ZARAGOZA' : 'ZARAGOZA',
           'ALICANTE' : 'ALICANTE',
           'CADIZ' : 'CADIZ'}
arr=list(df.nomprov.unique())
for name in arr:
    if(name not in city_name.keys()):
        city_name[name]="OTH"
        
arr=list(df_t.nomprov.unique())
for name in arr:
    if(name not in city_name.keys()):
        city_name[name]="OTH"
        
df.nomprov = df.nomprov.apply(lambda x: city_name.get(x,x)) # TRAIN

df_t.nomprov = df_t.nomprov.apply(lambda x: city_name.get(x,x)) # TEST

# **ENCODING AND MAKING OF DATAFRAMES**

In [ ]:
# MAKING OF df_train consisting of all feature columns from df dataframe. 

df_train = df.loc[:,'fecha_dato':'segmento']
df_train['cust_since']=df['cust_since']

# NOW WE DROP ANOTHER 6 COLUMNS BECAUSE THEY SHOWED NO IMPORTANCE IN OUR PLOT IMPORTANCE GRAPH.

drop=['ind_empleado','pais_residencia','indrel_1mes','indrel','indresi','indfall']
df_train.drop(columns=drop,inplace=True)
df_t.drop(columns=drop,inplace=True)

# NOW WE DO ONE-HOT ENCODING ON OUR FEATURE COLUMNS.

df_train=pd.get_dummies(df_train,columns=['nomprov','sexo','ind_nuevo','indext','canal_entrada','ind_actividad_cliente','segmento','tiprel_1mes'])

df_test=pd.get_dummies(df_t,columns=['nomprov','sexo','ind_nuevo','indext','canal_entrada','ind_actividad_cliente','segmento','tiprel_1mes'])

# df_y consists of all the target columns and few other important columns of which we find the lags and toggles.

df_y = df.loc[:,'ind_ahor_fin_ult1':'ind_recibo_ult1']
df_y['fecha_dato']=df['fecha_dato']
df_y['ncodpers']=df['ncodpers'] 
df_y['ind_actividad_cliente']=df['ind_actividad_cliente']
df_y['segmento']=df['segmento']


# NOW We have 3 dataframes df_train,df_test and df_y.

# **FEATURE ENGINEERING**

***TEST DATA***

**LAGS FOR TEST DATA**

In [ ]:
# We Have used few feature columns of lags and toggles.

# Lags - Lag of 1 month,Lag of 2 month and Lag of 12 month and Lag of segmento and ind_actividad_cliente.

# Toggle - Toggle of past 6 months.

# These are the lags and toggles which we will merge with test data.

# Lag of 1 month before.

df_1 = df[usecols2]
df_1 = df_1[df_train['fecha_dato']=="2016-04-28"]

# Lag of 2 months before.

df_2 = df[usecols2]
df_2 = df_2[df_train['fecha_dato']=="2016-03-28"]

# Lag of 12 months before.

df_12=df[usecols2]
df_12 = df_12[df_train['fecha_dato']=="2015-05-28"]

# Lag for ind_actividad_cliente from 6 months ago 

df_cliente=df[['fecha_dato','ncodpers','ind_actividad_cliente']]
df_cliente = df_cliente[df_train['fecha_dato']=="2015-11-28"]

# Lag for segmento from 6 months ago

df_segmento=df[['fecha_dato','ncodpers','segmento']]
df_segmento = df_segmento[df_train['fecha_dato']=="2015-11-28"]

**TOGGLE OF LAST 6 MONTHS**

In [ ]:
# Toggle of past 6 months ,We have 2 types of toggles -> 01_toggle and 10_toggle.

df_toggle=df[usecols2]
df_toggle=df_toggle[(df_train['fecha_dato']>="2015-10-28") & (df_train['fecha_dato']<="2016-04-28")]
for c in df_toggle.columns[2:]:
    df_toggle[c+'_01_toggle']=0
    df_toggle[c+'_10_toggle']=0
for c in usecols2[2:]:
    df_toggle.loc[(df_toggle.fecha_dato=="2016-04-28") & (df_toggle.ncodpers==df_toggle.ncodpers.shift(1) ) & (df_toggle[c]!=df_toggle[c].shift(1)) & (df_toggle[c]==1) ,c+'_01_toggle']=df_toggle[c+'_01_toggle']+1
    df_toggle.loc[(df_toggle.fecha_dato=="2016-04-28") & (df_toggle.ncodpers==df_toggle.ncodpers.shift(1) ) & (df_toggle[c]!=df_toggle[c].shift(1)) & (df_toggle[c]==0) ,c+'_10_toggle']=df_toggle[c+'_10_toggle']+1
    df_toggle.loc[(df_toggle.fecha_dato=="2016-04-28") & (df_toggle.ncodpers==df_toggle.ncodpers.shift(2) ) & (df_toggle[c].shift(1)!=df_toggle[c].shift(2)) & (df_toggle[c].shift(1)==1) ,c+'_01_toggle']=df_toggle[c+'_01_toggle']+1
    df_toggle.loc[(df_toggle.fecha_dato=="2016-04-28") & (df_toggle.ncodpers==df_toggle.ncodpers.shift(2) ) & (df_toggle[c].shift(1)!=df_toggle[c].shift(2)) & (df_toggle[c].shift(1)==0) ,c+'_10_toggle']=df_toggle[c+'_10_toggle']+1
    df_toggle.loc[(df_toggle.fecha_dato=="2016-04-28") & (df_toggle.ncodpers==df_toggle.ncodpers.shift(3) ) & (df_toggle[c].shift(2)!=df_toggle[c].shift(3)) & (df_toggle[c].shift(2)==1) ,c+'_01_toggle']=df_toggle[c+'_01_toggle']+1
    df_toggle.loc[(df_toggle.fecha_dato=="2016-04-28") & (df_toggle.ncodpers==df_toggle.ncodpers.shift(3) ) & (df_toggle[c].shift(2)!=df_toggle[c].shift(3)) & (df_toggle[c].shift(2)==0) ,c+'_10_toggle']=df_toggle[c+'_10_toggle']+1
    df_toggle.loc[(df_toggle.fecha_dato=="2016-04-28") & (df_toggle.ncodpers==df_toggle.ncodpers.shift(4) ) & (df_toggle[c].shift(3)!=df_toggle[c].shift(4)) & (df_toggle[c].shift(3)==1) ,c+'_01_toggle']=df_toggle[c+'_01_toggle']+1
    df_toggle.loc[(df_toggle.fecha_dato=="2016-04-28") & (df_toggle.ncodpers==df_toggle.ncodpers.shift(4) ) & (df_toggle[c].shift(3)!=df_toggle[c].shift(4)) & (df_toggle[c].shift(3)==0) ,c+'_10_toggle']=df_toggle[c+'_10_toggle']+1
    df_toggle.loc[(df_toggle.fecha_dato=="2016-04-28") & (df_toggle.ncodpers==df_toggle.ncodpers.shift(5) ) & (df_toggle[c].shift(4)!=df_toggle[c].shift(5)) & (df_toggle[c].shift(4)==1) ,c+'_01_toggle']=df_toggle[c+'_01_toggle']+1
    df_toggle.loc[(df_toggle.fecha_dato=="2016-04-28") & (df_toggle.ncodpers==df_toggle.ncodpers.shift(5) ) & (df_toggle[c].shift(4)!=df_toggle[c].shift(5)) & (df_toggle[c].shift(4)==0) ,c+'_10_toggle']=df_toggle[c+'_10_toggle']+1
    df_toggle.loc[(df_toggle.fecha_dato=="2016-04-28") & (df_toggle.ncodpers==df_toggle.ncodpers.shift(6) ) & (df_toggle[c].shift(5)!=df_toggle[c].shift(6)) & (df_toggle[c].shift(5)==1) ,c+'_01_toggle']=df_toggle[c+'_01_toggle']+1
    df_toggle.loc[(df_toggle.fecha_dato=="2016-04-28") & (df_toggle.ncodpers==df_toggle.ncodpers.shift(6) ) & (df_toggle[c].shift(5)!=df_toggle[c].shift(6)) & (df_toggle[c].shift(5)==0) ,c+'_10_toggle']=df_toggle[c+'_10_toggle']+1
drop=usecols2[2:]
df_toggle.drop(columns=drop,inplace=True)
df_toggle.fillna(0,inplace=True)
df_toggle=df_toggle[(df_train['fecha_dato']=="2016-04-28")]

**RAM MANAGEMENT**

In [ ]:
# We delete df and df_t for RAM Management .
del(df)
del(df_t)

***TRAIN DATA***

**REDUCING TIMEFRAME TO REQUIRED MONTHS**

In [ ]:
# We are using data of only 1 month that is April 2016.
# That is why we have taken only months which are needed for lags and toggles for April 2016.

df_train = df_train[((df_train['fecha_dato']=="2015-04-28"))  | ((df_train['fecha_dato']>="2015-09-28") & (df_train['fecha_dato']<="2016-04-28"))]
df_y = df_y[((df_y['fecha_dato']=="2015-04-28")) | ((df_y['fecha_dato']>="2015-09-28") & (df_y['fecha_dato']<="2016-04-28"))]

**ADDING TOGGLE DATA TO TRAIN**

In [ ]:
# Now we will do toggle for the past 6 months on train data.
# Even here we have 2 types of toggles 01_toggle and 10_toggle.

for c in df_y.columns[:24]:
    df_y[c+'_01_toggle']=0
    df_y[c+'_10_toggle']=0

for c in df_y.columns[:24]:
    df_y.loc[(df_y.fecha_dato=="2016-04-28") & (df_y.ncodpers==df_y.ncodpers.shift(2) ) & (df_y[c].shift(1)!=df_y[c].shift(2)) & (df_y[c].shift(1)==1),c+'_01_toggle']=df_y[c+'_01_toggle']+1
    df_y.loc[(df_y.fecha_dato=="2016-04-28") & (df_y.ncodpers==df_y.ncodpers.shift(2) ) & (df_y[c].shift(1)!=df_y[c].shift(2)) & (df_y[c].shift(1)==0),c+'_10_toggle']=df_y[c+'_10_toggle']+1
    df_y.loc[(df_y.fecha_dato=="2016-04-28") & (df_y.ncodpers==df_y.ncodpers.shift(3) ) & (df_y[c].shift(2)!=df_y[c].shift(3)) & (df_y[c].shift(2)==1),c+'_01_toggle']=df_y[c+'_01_toggle']+1
    df_y.loc[(df_y.fecha_dato=="2016-04-28") & (df_y.ncodpers==df_y.ncodpers.shift(3) ) & (df_y[c].shift(2)!=df_y[c].shift(3)) & (df_y[c].shift(2)==0),c+'_10_toggle']=df_y[c+'_10_toggle']+1
    df_y.loc[(df_y.fecha_dato=="2016-04-28") & (df_y.ncodpers==df_y.ncodpers.shift(4) ) & (df_y[c].shift(3)!=df_y[c].shift(4)) & (df_y[c].shift(3)==1),c+'_01_toggle']=df_y[c+'_01_toggle']+1
    df_y.loc[(df_y.fecha_dato=="2016-04-28") & (df_y.ncodpers==df_y.ncodpers.shift(4) ) & (df_y[c].shift(3)!=df_y[c].shift(4)) & (df_y[c].shift(3)==0),c+'_10_toggle']=df_y[c+'_10_toggle']+1
    df_y.loc[(df_y.fecha_dato=="2016-04-28") & (df_y.ncodpers==df_y.ncodpers.shift(5) ) & (df_y[c].shift(4)!=df_y[c].shift(5)) & (df_y[c].shift(4)==1),c+'_01_toggle']=df_y[c+'_01_toggle']+1
    df_y.loc[(df_y.fecha_dato=="2016-04-28") & (df_y.ncodpers==df_y.ncodpers.shift(5) ) & (df_y[c].shift(4)!=df_y[c].shift(5)) & (df_y[c].shift(4)==0),c+'_10_toggle']=df_y[c+'_10_toggle']+1
    df_y.loc[(df_y.fecha_dato=="2016-04-28") & (df_y.ncodpers==df_y.ncodpers.shift(6) ) & (df_y[c].shift(5)!=df_y[c].shift(6)) & (df_y[c].shift(5)==1),c+'_01_toggle']=df_y[c+'_01_toggle']+1
    df_y.loc[(df_y.fecha_dato=="2016-04-28") & (df_y.ncodpers==df_y.ncodpers.shift(6) ) & (df_y[c].shift(5)!=df_y[c].shift(6)) & (df_y[c].shift(5)==0),c+'_10_toggle']=df_y[c+'_10_toggle']+1
    df_y.loc[(df_y.fecha_dato=="2016-04-28") & (df_y.ncodpers==df_y.ncodpers.shift(7) ) & (df_y[c].shift(6)!=df_y[c].shift(7)) & (df_y[c].shift(6)==1),c+'_01_toggle']=df_y[c+'_01_toggle']+1
    df_y.loc[(df_y.fecha_dato=="2016-04-28") & (df_y.ncodpers==df_y.ncodpers.shift(7) ) & (df_y[c].shift(6)!=df_y[c].shift(7)) & (df_y[c].shift(6)==0),c+'_10_toggle']=df_y[c+'_10_toggle']+1

**REDUCING TIMEFRAME FURTHER FOR ONLY LAGS**

In [ ]:
# Now we reduce the timeframe even lesser for only the lags on train data.

df_train = df_train[((df_train['fecha_dato']=="2015-04-28"))  | (df_train['fecha_dato']=="2015-10-28") | ((df_train['fecha_dato']>="2016-02-28") & (df_train['fecha_dato']<="2016-04-28"))]
df_y = df_y[((df_y['fecha_dato']=="2015-04-28")) | (df_y['fecha_dato']=="2015-10-28") | ((df_y['fecha_dato']>="2016-02-28") & (df_y['fecha_dato']<="2016-04-28"))]

**LAGS ON TRAIN DATA**

In [ ]:
# Now we do lags on the train data.

for c in df_y.columns[:24]:
    df_y.loc[df_y.ncodpers==df_y.ncodpers.shift(1),c+'_lag1']=df_y[c].shift(1)
    df_y.loc[df_y.ncodpers==df_y.ncodpers.shift(2),c+'_lag2']=df_y[c].shift(2)
    df_y.loc[df_y.ncodpers==df_y.ncodpers.shift(4),c+'_lag12']=df_y[c].shift(4)

df_y.loc[df_y.ncodpers==df_y.ncodpers.shift(3),'ind_actividad_cliente'+'_lag']=df_y['ind_actividad_cliente'].shift(3)

df_y.loc[df_y.ncodpers==df_y.ncodpers.shift(3),'segmento'+'_lag']=df_y['segmento'].shift(3)

**REDUCING OUR TIMEFRAME TO ONLY THE TRAINING MONTH**

In [ ]:
# Now We reduce our timeframe only on our training month which is April 2016.

df_train = df_train[((df_train['fecha_dato']=="2016-04-28"))]

df_y = df_y[((df_y['fecha_dato']=="2016-04-28"))]

1. **MERGING THE LAGS AND TOGGLE DATA TO OUR TEST DATA**
2. **WE WANTED TO DO IT UP AT THE PLACE WHERE WE DEFINED THE DATAFRAMES FOR LAGS AND TOGGLE BUT WE WERE**
**FACING RAM ISSUES**

In [ ]:
# RENAMING OF LAG COLUMNS BEFORE MERGING INTO TEST DATA.

for c in df_1.columns[2:]:
    df_1.rename(columns={c:c+"_lag1"}, inplace=True)

for c in df_2.columns[2:]:
    df_2.rename(columns={c:c+"_lag2"},inplace=True)

for c in df_12.columns[2:]:
    df_12.rename(columns={c:c+"_lag12"},inplace=True)

df_cliente.rename(columns={'ind_actividad_cliente': 'ind_actividad_cliente' + '_lag'},inplace=True)

df_segmento.rename(columns={'segmento': 'segmento' + '_lag'},inplace=True)

# MERGING OF TOGGLE DATA AND LAG DATA INTO TEST DATA

df_test=df_test.merge(df_1.loc[:, 'ncodpers':], on='ncodpers', how='left')
df_test=df_test.merge(df_2.loc[:, 'ncodpers':], on='ncodpers', how='left')
df_test=df_test.merge(df_12.loc[:, 'ncodpers':], on='ncodpers', how='left')
df_test=df_test.merge(df_toggle.loc[:, 'ncodpers':], on='ncodpers', how='left')
df_test=df_test.merge(df_cliente.loc[:, 'ncodpers':], on='ncodpers', how='left')
df_test=df_test.merge(df_segmento.loc[:, 'ncodpers':], on='ncodpers', how='left')

**REMOVING NULL VALUES AND OTHER PROCESSING IN THE NEWLY ADDED COLUMNS**

In [ ]:
# NOW WE WILL FILL NULL DATA IN THE NEW COLUMNS ADDED IN TRAIN AND TEST DATA

# TRAIN

df_y.loc[df_y.ind_actividad_cliente_lag.isnull()==True,'ind_actividad_cliente_lag']=df_y.ind_actividad_cliente
df_y.loc[df_y.segmento_lag.isnull()==True,'segmento_lag']=df_y.segmento
df_y.fillna(0,inplace=True)

# TEST

df_test.loc[(df_test.ind_actividad_cliente_lag.isnull()==True) & (df_test.ind_actividad_cliente_0==1),'ind_actividad_cliente_lag']=0
df_test.loc[(df_test.ind_actividad_cliente_lag.isnull()==True) & (df_test.ind_actividad_cliente_1==1),'ind_actividad_cliente_lag']=1
df_test.loc[(df_test.segmento_lag.isnull()==True) & (df_test['segmento_01 - TOP']==1) ,'segmento_lag']='TOP'
df_test.loc[(df_test.segmento_lag.isnull()==True) & (df_test['segmento_02 - PARTICULARES']==1) ,'segmento_lag']='PARTICULARES'
df_test.loc[(df_test.segmento_lag.isnull()==True) & (df_test['segmento_03 - UNIVERSITARIO']==1) ,'segmento_lag']='UNIVERSITARIO'
df_test.fillna(0,inplace=True)

# CHANGING DATA TYPE OF NEW COLUMNS

for c in df_y.columns[28:149]:
    df_y[c]=df_y[c].astype("int8")

for c in df_test.columns[8:163]:
    df_test[c]=df_test[c].astype("int8")
    
# CONVERTING STRING DATA INTO NUMERICALS.

convert={"02 - PARTICULARES" : 1,
         "PARTICULARES" : 1,
         "03 - UNIVERSITARIO" : 2,
         "UNIVERSITARIO" : 2,
         "01 - TOP" : 3,
         "TOP" : 3}

df_y.segmento_lag = df_y.segmento_lag.apply(lambda x: convert.get(x,x))

df_test.segmento_lag = df_test.segmento_lag.apply(lambda x: convert.get(x,x))

**SOME FINAL WORK NEEDED TO DO BEFORE TRAINING**

In [ ]:
# SHIFTING OF ALL THE LAGS AND TOGGLES FROM df_y to df_train

for c in df_y.columns[26:]:
    df_train[c]=df_y[c]
    df_y.drop(columns=c, inplace=True)

# REALIGINING OF COLUMNS OF TRAIN AND TEST DATA

col=df_test.columns
df_train=df_train[col]

# MAKING OF THE result dataframe.

result=pd.DataFrame()
result['ncodpers']=df_test['ncodpers']

# DROPPING OF COLUMNS WHICH ARE NOT NEEDED IN TRAINING PHASE.

df_train.drop(columns=['fecha_dato','ncodpers', 'fecha_alta'], inplace=True)
df_test.drop(columns=['fecha_dato','ncodpers', 'fecha_alta'],inplace=True)
df_y.drop(columns=['fecha_dato'], inplace=True)

# **TRAINING**

**MODEL SELECTION**

In [ ]:
# MODEL USED FOR THIS PART OF TRAINING

clf = CatBoostClassifier(iterations=300,
                        learning_rate=0.035,
                        depth=12,
                        loss_function='Logloss',
                        min_data_in_leaf=25,
                        rsm=0.42)

**FINAL TRAINING**

In [ ]:
# TRAINING FOR OUR DATA

for c in df_y.columns[:-1]:
    y_train = df_y[c]
    clf.fit(df_train, y_train)
    p_train=clf.predict_proba(df_test)[:,1]
    result[c]=p_train
    print(c)

In [ ]:
result.to_csv('result_old.csv',index=False)